In [1]:
import os
from fnmatch import fnmatch
from trainData_featExt_glcm import trainData_featExt_glcm
import numpy as np
from PIL import Image
from filtering.filters import Median
from feature_extraction.glcm import Glcm
from feature_extraction.gabor import Gabor
from sklearn import svm
import sklearn
import time
from sklearn import preprocessing
import pickle

In [ ]:
#Read features of training samples
train_samples_feats_matrix_GLCM = np.load("train_samples_feats_matrix_GLCM.npy")

In [ ]:
#Load features of validation samples
valid_samples_feats_matrix_GLCM = np.load("valid_samples_feats_matrix_GLCM.npy")

In [2]:
test_samples_feats_matrix_GLCM = np.load("test_samples_feats_matrix_GLCM.npy")

In [8]:
print test_samples_feats_matrix_GLCM

[[  8.93458933e-02   2.35035211e-01   1.94728241e-01 ...,   2.36283131e-05
    2.07620139e-05   2.49394241e-05]
 [  1.56913962e-01   2.32400757e-01   1.52913882e-01 ...,   4.00539066e-05
    4.25214508e-05   3.94451985e-05]
 [  1.40374228e-01   1.98424296e-01   1.49077918e-01 ...,   4.91008804e-05
    4.00991173e-05   4.09880328e-05]
 ..., 
 [  1.11267827e-01   1.46887882e-01   1.24380977e-01 ...,   3.52089612e-05
    3.10692812e-05   3.35108581e-05]
 [  1.42270071e-01   2.31652246e-01   1.73836788e-01 ...,   3.42573591e-04
    3.40316535e-04   3.65707206e-04]
 [  1.79629374e-01   2.42687798e-01   1.61647567e-01 ...,   4.16565901e-04
    4.14679822e-04   4.22613345e-04]]


In [ ]:
#Train svn model from training data
t0 = time.time()
svm_model = svm.OneClassSVM(kernel='rbf', gamma=0.1, nu=0.1)
svm_model.fit(train_samples_feats_matrix_GLCM)
print("traing time: ", time.time() - t0)

In [3]:
svm_model = pickle.load( open( "SVM_GLCM_[0.2, 0.01]", "rb" ))

In [4]:
#Ground truth
y1 = np.repeat(+1, 6000) #normal
y2 = np.repeat(-1, 4000) #abnormal
y = np.concatenate((y1,y2))

In [ ]:
#Predict labels of training dataset
y_predict_train = svm_model.predict(train_samples_feats_matrix_GLCM)
print ("training errors", y_predict_train[y_predict_train == -1].size)

In [5]:
#Predict labels of new data points
y_predict = svm_model.predict(test_samples_feats_matrix_GLCM)
print ("number of samples detected as outliers", y_predict[y_predict == -1].size)
print ("detected as normal", y_predict[y_predict == +1].size)

('number of samples detected as outliers', 5411)
('detected as normal', 4589)


In [6]:
y_predict_score = svm_model.decision_function(test_samples_feats_matrix_GLCM)

In [ ]:
print y_predict_score

In [7]:
f1_binary = sklearn.metrics.f1_score(y, y_predict, pos_label =-1, average = 'binary')
f1_macro = sklearn.metrics.f1_score(y, y_predict, average = 'macro')
auc = sklearn.metrics.roc_auc_score(y, y_predict_score)
Math_Cof = sklearn.metrics.matthews_corrcoef(y, y_predict) 
tn, fp, fn, tp  = sklearn.metrics.confusion_matrix(y, y_predict).ravel()
print ("f1_binary: ", f1_binary, "f1_macro: ", f1_macro, "MAth_cof: ", Math_Cof , " auc: ", auc)
print("confusion matrix", tn, fp, fn, tp)

('f1_binary: ', 0.47370098820529172, 'f1_macro: ', 0.50297571839200272, 'MAth_cof: ', 0.026462392168469483, ' auc: ', 0.52542170833333335)
('confusion matrix', 2229, 1771, 3182, 2818)


In [ ]:
import matplotlib.pyplot as pl
import matplotlib
from skimage import io

fpr,tpr,thresh = sklearn.metrics.roc_curve(y,y_predict_score)

f,ax = pl.subplots(1,1)
ax.plot(fpr,tpr,label="SVM")
ax.set_xlabel("False Positive Rate")
ax.set_ylabel("True Positive Rate")
ax.set_title("ROC curves")
ax.legend(loc="lower right")
# these are matplotlib.patch.Patch properties
props = dict(boxstyle='square', facecolor='white', alpha=0.5)

textstr = '$F1-binary=%.2f$\n$F1-macro=%.2f$\n$math-corcoeffient=%.2f$\n$AUC=%.2f$'%(0.6, 0.65, 0.31, 0.7)
# place a text box in upper left in axes coords
ax.text(0.95, 0.2, textstr, transform=ax.transAxes, fontsize=14,
    horizontalalignment='right', verticalalignment='bottom', bbox=props)

io.show()

In [ ]:
f.savefig('SVM_GLCM.png')